The task here is to do a part of data preprocessing before connecting it to the **PowerBI** environment.

Basically, I'll do next things:
- Create **Kaggle losses** .csv file (will be demonstated in one dashboard).
- Create .csv file, which contains all product families (will be used to comfortly establish relationships).
- Modify train dataset by removing observations when stores didn't start operating (otherwise it will be pretty rough to do inside Power Query).
- Modify holiday dataset (make terms more understandable and preserve only 1 (most important) holiday per day (to maintain one-to-many relationships).
- Resample and interpolate oil dataset.

In [1]:
import pandas as pd

In [10]:
losses_dict = {
    'Holt-Winters': 0.43131,
    'Ridge': 0.45105,
    'Random Forest': 0.42014,
    'k-NN': 0.43540,
    'XGBoost': 0.43196,
    'LightGBM': 0.42780,
    'Ensemble': 0.39999
}

model_losses = pd.Series(losses_dict)

In [13]:
model_losses.to_csv('preprocessed_datasets/model_losses.csv', index_label='Model', header=['RMSLE Loss'])

In [2]:
train = pd.read_csv('initial_datasets/train.csv', index_col=0)
starting_day = pd.read_csv('preprocessed_datasets/starting_day.csv', index_col=0).squeeze()

unique_sf = train.groupby(['store_nbr', 'family']).count().index.values

In [9]:
pd.Series(train['family'].unique()).to_csv('preprocessed_datasets/families.csv', header=['Family'])

In [30]:
for s, f in unique_sf:
    temp = train[(train['store_nbr'] == s) & (train['family'] == f)]
    indexes_to_drop = temp.iloc[:starting_day[s], :].index

    train = train.drop(index=indexes_to_drop)

In [48]:
grouped_ds_jan1 = train[(train['date'].dt.month == 1) & (train['date'].dt.day == 1)].groupby(['date', 'store_nbr']).sales.sum()

grouped_ds_jan1 = grouped_ds_jan1[grouped_ds_jan1 == 0].reset_index()

to_delete = pd.merge(train.reset_index(), grouped_ds_jan1, on=['date', 'store_nbr'], how='inner')
train = train.drop(to_delete.id)

In [49]:
train.to_csv('preprocessed_datasets/train_modified_start.csv')

In [3]:
holidays = pd.read_csv('initial_datasets/holidays_events.csv', index_col=0, parse_dates=True)
holidays['priority'] = holidays['locale'].map({'National': 1, 'Regional': 2, 'Local': 3})
holidays = holidays.sort_values(['priority'])
holidays = holidays[~holidays.index.duplicated()]

In [4]:
holidays.loc[holidays[holidays['transferred'] == True].index, 'type'] = 'Moved'
holidays['type'] = holidays['type'].map(lambda x: 'Sat/Sun Working Day' if x == 'Work Day' else x)
holidays = holidays.drop(columns=['transferred', 'priority'])

min_date = '2013-01-01'
max_date = '2017-08-31'

complete_dates = pd.date_range(start=min_date, end=max_date)
holidays = holidays.reindex(complete_dates)

In [5]:
holidays = holidays.fillna('Working Day')

holidays.to_csv('preprocessed_datasets/holidays_events_modified.csv', index_label='date')

In [5]:
oil = pd.read_csv('initial_datasets/oil.csv', index_col=0, parse_dates=True)
oil = oil.resample('1D').asfreq().interpolate('linear').reset_index()
oil.to_csv('preprocessed_datasets/oil_modified.csv', index=False)